# what is this?
Experimneting with data augmentation for NLP. The goal is to experiment with the dataaugmentation to increase training size of teh original data.

## install required libraries

In [1]:
import random

In [2]:
!pip install fastprogress

## import all

In [3]:
import pandas as pd
import nltk
import spacy
import multiprocessing as mp
from typing import List
from functools import partial
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from pandas.core.frame import DataFrame
from fastai.text.all import *
from fastprogress import progress_bar
from fastcore.all import *

## Getting the data

In [4]:
df_train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
df_train.head(5)

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, ""All right, Dolly, you may go.""\n""Oh, mother!"" Dolly cried, overwhelmed with sudden delight. ""Really?\nOh, I am so glad! Are you sure you're willing?""\n""I've persuaded myself to be willing, against my will,"" returned Mrs. Fayre, whimsically...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as quickly as it came, and two days after their sleigh ride there was scarcely a vestige of white on the ground. Tennis was again possible and a great game was in progress on the court at Pine Laurel. Patty and Roger were playing against Elise and Sam Blaney, and the pairs were well matched.\nBut the long-contested victory finally went against Patty, and she laughingly accepted defeat.\n""Only because Patty's not quite back on her game yet,"" Roger defended; ""this child has been on the sick list, you know, Sam, and she isn't up to her own mark.""\n""We...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,"And outside before the palace a great garden was walled round, filled full of stately fruit-trees, gray olives and sweet figs, and pomegranates, pears, and apples, which bore the whole year round. For the rich south-west wind fed them, till pear grew ripe on pear, fig on fig, and grape on grape, all the winter and the spring. And at the farther end gay flower-beds bloomed through all seasons of the year; and two fair fountains rose, and ran, one through the garden grounds, and one beneath the palace gate, to water all the town. Such noble gifts the heavens had given to Alcinous the wise.\n...",-1.054013,0.450007
4,37c1b32fb,NaN,NaN,"Once upon a time there were Three Bears who lived together in a house of their own in a wood. One of them was a Little, Small, Wee Bear; and one was a Middle-sized Bear, and the other was a Great, Huge Bear. They had each a pot for their porridge; a little pot for the Little, Small, Wee Bear; and a middle-sized pot for the Middle Bear; and a great pot for the Great, Huge Bear. And they had each a chair to sit in; a little chair for the Little, Small, Wee Bear; and a middle-sized chair for the Middle Bear; and a great chair for the Great, Huge Bear. And they had each a bed to sleep in; a li...",0.247197,0.510845


use wordnet to repalce random word with their synonyms

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Creating helpers
creating helpers to find synonyms and replace them with picked words

quick check to see how the internals works

In [6]:
synsets = wn.synsets("leave", pos="v")
lemmas = synsets[0].lemmas()
lemmas[1].name()

'go_forth'

In [7]:
spacy_doc = "i am a cat"
" ".join([spacy_doc[:3], "awesome", spacy_doc[1 + 2 :]])

'i a awesome m a cat'

In [8]:
text = "As Roger had predicted, the snow departed"
L(text.split())

(#7) ['As','Roger','had','predicted,','the','snow','departed']

In [9]:
wn.synsets?

In [10]:
text2 = "Apple is looking at buying U.K. startup for $1 billion"
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for token in doc:
    print(token.text, token.pos_)

As SCONJ
Roger PROPN
had AUX
predicted VERB
, PUNCT
the DET
snow NOUN
departed VERB


In [11]:
def tokenify(text: str):
    """Split text into a list with optional rules"""
    #text = text.lower()
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return {tok.text:tok.pos_ for tok in doc}
    #return L(tok.text for tok in doc)
    
def get_synonyms(token, pos=None):
    """Get the synonyms of word from Wordnet."""
    lemmas = set().union(*[s.lemmas() for s in wn.synsets(token, pos=pos)])
    return list(set(l.name().lower().replace("_", " ") for l in lemmas) - {token})   

In [12]:
tokenify(text)

{'As': 'SCONJ',
 'Roger': 'PROPN',
 'had': 'AUX',
 'predicted': 'VERB',
 ',': 'PUNCT',
 'the': 'DET',
 'snow': 'NOUN',
 'departed': 'VERB'}

creating transformation functions to replace adjectives, nouns, verbs with synonyms

In [13]:
text.index("predicted")

13

## Write transformation functions

### Replace with synonym

In [14]:
def replace_tok_with_synonym(text: str, pos="v"):
    """replace a random verb with a synonym."""
    pos_dict = {"v": "VERB", "a": "ADJ", "n": "Noun"}
    tokens = tokenify(text)
    pos_words = L(tok for tok in tokens.keys() if tokens[tok] == pos_dict[pos])
    idx = random.choice(range(len(pos_words))) if pos_words else None
    synonyms = get_synonyms(pos_words[idx], pos=pos) if idx else None
    if synonyms:
        return text.replace(pos_words[idx], synonyms[0])
    else:
        return None

### Random deletion
Code taken directly from the paper and the related repo [here](https://github.com/jasonwei20/eda_nlp/blob/master/code/eda.py)

In [15]:
a = {"a":1, "b":2}
list(a.keys())

['a', 'b']

In [16]:
def random_deletion(text: str, p: float= 0.2):
    """Delete random tokens from a text"""
    tokens = list(tokenify(text).keys())
    if len(tokens) == 1: return tokens
    new_tokens = []
    
    for token in tokens:
        r = random.uniform(0, 1)
        if r > p:
            new_tokens.append(token)

    #if you end up deleting all words, just return a random word
    if len(new_tokens) == 0:
        rand_int = random.randint(0, len(tokens)-1)
        return [tokens[rand_int]]

    return ' '.join(new_tokens)

def erase(p: float= 0.2):
    return partial(random_deletion, p = p)

### Random swap

In [17]:
def swap(tokens: List, threshold: int):
    """swap two tokens"""
    idx1 = random.randint(0, len(tokens)-1)
    idx2 = idx1
    counter = 0
    while idx2 == idx1:
        idx2 = random.randint(0, len(tokens)-1)
        counter += 1
        if counter > threshold:
            return tokens
        tokens[idx1], tokens[idx2] =  tokens[idx2], tokens[idx1]
        
    return tokens

def random_swap(text: str,
                threshold: int= 3):
    """swap random tokens in a text
    and return the transformed text"""
    tokens = list(tokenify(text).keys())
    n_times = max(1, len(tokens))
    for _ in range(n_times):
        tokens = swap(tokens, threshold)
    return ' '.join(tokens)

def swapabble(threshold:int = 3):
    return partial(random_swap,
                   threshold=threshold)


In [18]:
random_swap("hello there, how are you")

'there , hello you how are'

quick test

In [19]:
sample_text = df_train.excerpt.loc[0]
sample_text

'When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes.'

In [20]:
replace_tok_with_synonym(sample_text)

In [21]:
replace_tok_with_synonym(sample_text, "n")

In [22]:
df_tiny = df_train.loc[:3]
df_tiny

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, ""All right, Dolly, you may go.""\n""Oh, mother!"" Dolly cried, overwhelmed with sudden delight. ""Really?\nOh, I am so glad! Are you sure you're willing?""\n""I've persuaded myself to be willing, against my will,"" returned Mrs. Fayre, whimsically...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as quickly as it came, and two days after their sleigh ride there was scarcely a vestige of white on the ground. Tennis was again possible and a great game was in progress on the court at Pine Laurel. Patty and Roger were playing against Elise and Sam Blaney, and the pairs were well matched.\nBut the long-contested victory finally went against Patty, and she laughingly accepted defeat.\n""Only because Patty's not quite back on her game yet,"" Roger defended; ""this child has been on the sick list, you know, Sam, and she isn't up to her own mark.""\n""We...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,"And outside before the palace a great garden was walled round, filled full of stately fruit-trees, gray olives and sweet figs, and pomegranates, pears, and apples, which bore the whole year round. For the rich south-west wind fed them, till pear grew ripe on pear, fig on fig, and grape on grape, all the winter and the spring. And at the farther end gay flower-beds bloomed through all seasons of the year; and two fair fountains rose, and ran, one through the garden grounds, and one beneath the palace gate, to water all the town. Such noble gifts the heavens had given to Alcinous the wise.\n...",-1.054013,0.450007


In [23]:
#df_tiny["excerpt"] = df_tiny["excerpt"].transform(replace_tok_with_synonym,"n")

In [24]:
df_tiny

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, ""All right, Dolly, you may go.""\n""Oh, mother!"" Dolly cried, overwhelmed with sudden delight. ""Really?\nOh, I am so glad! Are you sure you're willing?""\n""I've persuaded myself to be willing, against my will,"" returned Mrs. Fayre, whimsically...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as quickly as it came, and two days after their sleigh ride there was scarcely a vestige of white on the ground. Tennis was again possible and a great game was in progress on the court at Pine Laurel. Patty and Roger were playing against Elise and Sam Blaney, and the pairs were well matched.\nBut the long-contested victory finally went against Patty, and she laughingly accepted defeat.\n""Only because Patty's not quite back on her game yet,"" Roger defended; ""this child has been on the sick list, you know, Sam, and she isn't up to her own mark.""\n""We...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,"And outside before the palace a great garden was walled round, filled full of stately fruit-trees, gray olives and sweet figs, and pomegranates, pears, and apples, which bore the whole year round. For the rich south-west wind fed them, till pear grew ripe on pear, fig on fig, and grape on grape, all the winter and the spring. And at the farther end gay flower-beds bloomed through all seasons of the year; and two fair fountains rose, and ran, one through the garden grounds, and one beneath the palace gate, to water all the town. Such noble gifts the heavens had given to Alcinous the wise.\n...",-1.054013,0.450007


In [25]:
random_deletion("i am a superhero, but...", 0.2)

'i am a superhero but ...'

In [26]:
#this gives error
#erase("i am a superhero, but...")

In [27]:
erase = erase()
erase("i am a superhero, but...")

'i am a superhero , but ...'

creating separate functions for verbs adjectives and nouns

In [28]:
def replace_verb(text: str):
    """replace verb with it's synonym"""
    return replace_tok_with_synonym(text)

def replace_noun(text: str):
    """replace noun with it's synonym"""
    return replace_tok_with_synonym(text, "n")

def replace_adj(text: str):
    """replace adjective with it's synonym"""
    return replace_tok_with_synonym(text, "a")

In [29]:
replace_verb("I predicted this would happen sometime")

'I predicted this would pass sometime'

quick test

In [30]:
df_tiny2 = df_train.loc[:1]
df_tiny2

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, ""All right, Dolly, you may go.""\n""Oh, mother!"" Dolly cried, overwhelmed with sudden delight. ""Really?\nOh, I am so glad! Are you sure you're willing?""\n""I've persuaded myself to be willing, against my will,"" returned Mrs. Fayre, whimsically...",-0.315372,0.480805


# Designing transform fucntion
* Should apply functions to the dataframe one by one
* After applying each function it should concatenate the transformed dataframe with the original and then return a new dataframe.
  * While doing so, it should remove the rows for which the text field was returned none

In [31]:
def transform(df: DataFrame, col:str, f):
    df_tf = df.copy()
    df_tf[col] = df_tf[col].transform(f)
    df_tf.dropna(inplace=True, subset=['excerpt'])
    return df_tf

In [32]:
sample_df = df_train.loc[:5]
sample_df.shape

(6, 6)

In [33]:
sample1= transform(df_tiny2, 'excerpt', replace_verb)

In [34]:
sample2 = transform(df_tiny2, 'excerpt', partial(random_deletion,p=0.2))

In [35]:
sample3 = transform(df_tiny2, 'excerpt', erase)

In [36]:
sample4 = transform(df_tiny2, 'excerpt', swapabble(threshold=3))

In [37]:
sample1.shape, sample2.shape, sample3.shape, sample4.shape

((1, 6), (2, 6), (2, 6), (2, 6))

In [38]:
sample1

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009


In [39]:
sample2

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned ballroom , it presented changed appearance . Instead of an interior scene winter \n The with - white canvas not laid on smoothly but over bumps and hillocks like palms evergreens that had decorated room were strewn tufts cotton diamond been lightly sprinkled them glittering crystal hung branches At end wall beautiful bear skin rug These for one girls And this gathered other called North South Each player given small flag which they plant reaching would have matter traveller obliged wear snowshoes",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time , Mrs. Fayre was somewhat her eyes Dolly with a uncertain expression . wanted to the child she craved but had hard work bring point of overcoming own objections \n At last however when meal nearly over smiled at daughter and said right may go Oh mother cried overwhelmed sudden delight Really ? I am glad Are sure 're willing 've persuaded against will returned whimsically confess just hate ca n't bear deprive trip And as say it would also keep Dotty home think in angel You blessed How are flew around hug that suffocated",-0.315372,0.480805


In [40]:
sample3

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the people returned to ballroom , it presented a decidedly changed . Instead of scene was winter landscape \n The floor covered with snow - white canvas not laid on smoothly but rumpled over bumps and hillocks real evergreens that had room were powdered flour strewn cotton Also diamond dust sprinkled them glittering crystal icicles hung from branches At each beautiful bear skin rug These rugs prizes boys And this game gathered at other called North Pole South Each player flag which they plant This would have easy matter traveller obliged wear snowshoes",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through time , Mrs. Fayre somewhat silent her eyes resting on Dolly a wistful uncertain expression . She to give the she craved had hard bring point of overcoming objections \n At however when nearly smiled little daughter said "" right may go Oh mother ! cried overwhelmed sudden delight Really ? I am so glad Are willing 've persuaded myself be my will returned whimsically just hate have ca deprive trip And as say it would also keep Dotty home altogether think shall in angel blessed lady How good are flew table gave hug that suffocated",-0.315372,0.480805


In [41]:
sample4

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When decidedly rug at returned obliged from beautiful flour Each crystal the and The - not boys sprinkled interior wear was . landscape evergreens that ballroom an wall snow changed girls canvas but white covered powdered numerous rumpled plant it Also called like small floor hillocks palms end other bumps decorated room strewn dust player icicles have had a on prizes reaching presented would them field people skin snowshoes they cotton This each traveller At smoothly this branches over These been for hung with lightly glittering And easy diamond gathered \n one bear North matter scene , t...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,""" All dinner daughter be Really blessed eyes trip whimsically Mrs. ? said also Dolly my with of bear that just She wanted nearly \n hate when persuaded sudden craved silent resting , work expression herself At myself uncertain angel through and How bring n't child objections sure shall And may little good hug you so Fayre go willing time keep flew ! cried pleasure the delight was however wistful lady mother over Are are 're I deprive a own she to altogether 've returned but ca Oh meal somewhat against Dotty right overcoming at around as gave it on in home suffocated glad say smiled overwhe...",-0.315372,0.480805


## Multiprocessing
Parallelly run all the diffeerent varient of the transform function i.e. with different parameters and then concatenate all results.

list of parameter variants

In [42]:
erase = erase
swapabble = swapabble()
args = [replace_verb, 
        replace_noun, 
        replace_adj,
        erase,
       swapabble]

collection callback

In [43]:
results = []
def collect_result(result):
    results.append(result)

In [44]:
n_workers=mp.cpu_count()
pause=0

creating partialtransform

In [45]:
tfm = partial(transform, sample_df, 'excerpt')

parallel processing with single function

In [46]:
#result[0].shape

In [47]:
#result[0].head()

parallel processing with multiple functions

In [48]:
#with pool(n_workers) as ex:
#    result2 = ex.map(tfm, [replace_verb, replace_noun, replace_adj])

In [49]:
#len(result2)

In [50]:
#result2

## The augmenter
* Take the transform function and run parallely on multiple replace functions.
* Collect results and do post cleanup.
* Put in a single dataframe

In [51]:
progress_bar?

In [52]:
def augmenter(df: DataFrame, col: str, tmflist: List) -> DataFrame:
    n_workers=mp.cpu_count()
    partial_tmf = partial(transform, df, col)
    r = parallel(partial_tmf, tmflist, progress=True)
    return pd.concat([df]+r, ignore_index=True)

In [53]:
sample_df.shape

(6, 6)

In [54]:
final_df = augmenter(sample_df, 'excerpt', args)

In [55]:
final_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, ""All right, Dolly, you may go.""\n""Oh, mother!"" Dolly cried, overwhelmed with sudden delight. ""Really?\nOh, I am so glad! Are you sure you're willing?""\n""I've persuaded myself to be willing, against my will,"" returned Mrs. Fayre, whimsically...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as quickly as it came, and two days after their sleigh ride there was scarcely a vestige of white on the ground. Tennis was again possible and a great game was in progress on the court at Pine Laurel. Patty and Roger were playing against Elise and Sam Blaney, and the pairs were well matched.\nBut the long-contested victory finally went against Patty, and she laughingly accepted defeat.\n""Only because Patty's not quite back on her game yet,"" Roger defended; ""this child has been on the sick list, you know, Sam, and she isn't up to her own mark.""\n""We...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,"And outside before the palace a great garden was walled round, filled full of stately fruit-trees, gray olives and sweet figs, and pomegranates, pears, and apples, which bore the whole year round. For the rich south-west wind fed them, till pear grew ripe on pear, fig on fig, and grape on grape, all the winter and the spring. And at the farther end gay flower-beds bloomed through all seasons of the year; and two fair fountains rose, and ran, one through the garden grounds, and one beneath the palace gate, to water all the town. Such noble gifts the heavens had given to Alcinous the wise.\n...",-1.054013,0.450007
4,37c1b32fb,NaN,NaN,"Once upon a time there were Three Bears who lived together in a house of their own in a wood. One of them was a Little, Small, Wee Bear; and one was a Middle-sized Bear, and the other was a Great, Huge Bear. They had each a pot for their porridge; a little pot for the Little, Small, Wee Bear; and a middle-sized pot for the Middle Bear; and a great pot for the Great, Huge Bear. And they had each a chair to sit in; a little chair for the Little, Small, Wee Bear; and a middle-sized chair for the Middle Bear; and a great chair for the Great, Huge Bear. And they had each a bed to sleep in; a li...",0.247197,0.510845
5,f9bf357fe,NaN,NaN,"Hal and Chester found ample time to take an inventory of the general's car. It was a huge machine, and besides being fitted up luxuriously was also furnished as an office, that the general might still be at work while he hurried from one part of the field to another when events demanded his immediate presence. Even now, with treachery threatening, and whirling along at a terrific speed, General Joffre, probably because of habit, fell to work sorting papers, studying maps and other drawings.\nFor almost two hours the car whirled along at top speed, and at length pulled up in the rear of an ...",-0.861809,0.480936
6,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decide

In [56]:
final_df.shape

(28, 6)

# Augmenting the full dataset
Generating excerpts by replacing noun, verb and adjective with their synonyms if any.

In [57]:
aug_df = augmenter(df_train, 'excerpt', args)

checking the augmented dataframe's shape

In [58]:
aug_df.shape

(13135, 6)

checking a couple of data

In [59]:
aug_df.head(2)

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,"When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThe...",-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewhat silent, her eyes resting on Dolly with a wistful, uncertain expression. She wanted to give the child the pleasure she craved, but she had hard work to bring herself to the point of overcoming her own objections.\nAt last, however, when the meal was nearly over, she smiled at her little daughter, and said, ""All right, Dolly, you may go.""\n""Oh, mother!"" Dolly cried, overwhelmed with sudden delight. ""Really?\nOh, I am so glad! Are you sure you're willing?""\n""I've persuaded myself to be willing, against my will,"" returned Mrs. Fayre, whimsically...",-0.315372,0.480805


In [60]:
aug_df.tail(3)

,id,url_legal,license,excerpt,target,standard_error
13132,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_Elements/Liquids,CC BY-SA 3.0,"n't second too cube into found states water part leave liquid . same Liquids is you things can state a ? around gas until characteristic container The are gravity blood variety with solid ( some called That have up Putting we hard discuss compounds to usually What Solids do Water an materials , be solution first \n One - change bubbles hold it made and will also because floating then in becomes take the surface If rest top of pour flat between middle Your cup ice Gases bottom fills shape They fill wo ; It that ) matter",0.255209,0.483866
13133,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Elementary_School/Solids,CC BY-SA 3.0,"dealing : Great figures solid HEE Points not these height – need life , sum of coplanar measurements up and length our the Solids These They daily vertices form width . are When drah later on ee which Your colinear polehedron about ; solids called is must angles useful two plural bars them with A lot only corners you know shapes - how The ) dron go edge actually polyhedra(pol three or plane very have faces Pyramids that chocolate things what will make total big like there a area polyhedra detail to straight into we volume means edges touch surfaces surface polyhedron(pol be former We \n la...",-0.215279,0.514128
13134,5b990ba77,https://en.wikibooks.org/wiki/Wikijunior:Biology/Kingdoms/Animals,CC BY-SA 3.0,"Only oceans cows birds Invertebrates inside can example move amphibians few quite animal further two ground all spiders cells least dig reptiles so ; Most Vertebrates swim vertebrates a , while not horses \n digest though the earth worms frogs brains cats of be sky at made animals invertebrates do They . in ) on for fly compared sponges mammals into many found groups over eat insects and subgroups — fish divided have type like thirty dogs crabs mollusks So There different kinds even five several arthropods other are Humans Animals main to jellyfish ( them backbone things",0.300779,0.512379


# Saving the augmented dataframe

In [61]:
aug_df.to_csv('./aug_df.csv')